In [1]:
import tensorflow as tf
import numpy as np
import matplotlib
import os
from sklearn.preprocessing import MinMaxScaler

# 그래프 외부에 출력
%matplotlib tk

tf.set_random_seed(777)  # 어느 컴퓨터에서 이 코드를 실행해도 학습 방향이 같도록, 다시 수행해도 같도록

if "DISPLAY" not in os.environ:
    # remove Travis CI Error
    matplotlib.use('Agg')

import matplotlib.pyplot as plt

In [2]:
# train Parameters
# seq_length = 4
seq_length = 4
data_dim = 6
hidden_dim = 10
output_dim = 1
learning_rate = 0.01
iterations = 892

scaler = MinMaxScaler(feature_range=(0,1)) # 데이터 일반화
scaler2 = MinMaxScaler(feature_range=(0, 1)) # 데이터 일반화

# xy = np.loadtxt('./v3data/train_v3_data_cct.csv', delimiter=',')
# cct, cas_swr, 446to477, uvb, ptmt (892행 5열)
xy = np.loadtxt('./v6data/train_v6_data.csv', delimiter=',')

x = scaler.fit_transform(xy[:, [0, 1, 3, 4, 5, 6]]) # x = 맨 마지막 ptmt 제외 모든 것
y = scaler2.fit_transform(xy[:, [2]]) 

# build a dataset
dataX = []
dataY = []

# print(len(x))
# print(len(y))
# print(x[0])
# print(y[0])

for i in range(0, len(x) - seq_length): # 한 행씩 dataX, Y에 추가
    _x = x[i:i + seq_length]
    _y = y[i + seq_length]  # Next close price
    dataX.append(_x)
    dataY.append(_y)
#     print(np.shape(_x))

print(np.shape(dataX))
print(np.shape(dataY))

date = '2018-04-21'
# xy2 = np.loadtxt('./v3data/'+ data +'.csv',delimiter=',')
# xy2 = np.loadtxt('./v3data/test_v3_data_cct_180331.csv', delimiter=',')
xy2 = np.loadtxt('./v6data/new/20180421.csv', delimiter=',') # 예측할 날짜
# xy2 = np.loadtxt('./v6data/test_v6_data_180331.csv', delimiter=',')

# x2 = scaler.fit_transform(xy2) # 데이터 없을때 (데이터란 ptmt 값)
x2 = scaler.fit_transform(xy2[:, [0, 1, 3, 4, 5, 6]]) # 데이터 있을때
y2 = scaler2.fit_transform(xy2[:, [2]]) #데이터 있을때

# print(np.shape(x2))

dataX2 = []
dataY2 = [] #데이터 있을때

for i in range(0, len(x2) - seq_length):
    _x2 = x2[i:i+seq_length]
    _y2 = y2[i+seq_length] #데이터 있을때
    dataX2.append(_x2)
    dataY2.append(_y2) #데이터 있을때
# train/test split

train_size = len(dataX)
test_size = len(dataX2)

# print(train_size)
# print(test_size)

trainX, testX = np.array(dataX[0:train_size]),np.array(dataX2[0:test_size])
trainY = np.array(dataY[0:train_size])

# print(np.shape(trainX))

(3564, 4, 6)
(3564, 1)


In [3]:
# input place holders
X = tf.placeholder(tf.float32, [None, seq_length, data_dim])
Y = tf.placeholder(tf.float32, [None, 1])

# build a LSTM network
cell = tf.contrib.rnn.BasicLSTMCell(
    num_units=hidden_dim, state_is_tuple=True, activation=tf.tanh)
outputs, _states = tf.nn.dynamic_rnn(cell, X, dtype=tf.float32)
Y_pred = tf.contrib.layers.fully_connected(
    outputs[:, -1], output_dim, activation_fn=None)  # We use the last cell's output

# cost/loss
loss = tf.reduce_sum(tf.square(Y_pred - Y))  # sum of the squares
# optimizer
optimizer = tf.train.AdamOptimizer(learning_rate)
train = optimizer.minimize(loss)

Instructions for updating:
Use the retry module or similar alternatives.


In [4]:
with tf.Session() as sess:
    init = tf.global_variables_initializer()
    sess.run(init)

    # Training step
    for i in range(iterations):
        _, step_loss = sess.run([train, loss], feed_dict={
                                X: trainX, Y: trainY})
        print("[step: {}] loss: {}".format(i, step_loss))
    # Test step
    test_predict = sess.run(Y_pred, feed_dict={X: testX})
    
    test_predict = scaler2.inverse_transform(test_predict)
    dataY2 = scaler2.inverse_transform(dataY2) # 데이터 있을때

[step: 0] loss: 371.25750732421875
[step: 1] loss: 201.5220947265625
[step: 2] loss: 97.53817749023438
[step: 3] loss: 48.93126678466797
[step: 4] loss: 44.872840881347656
[step: 5] loss: 63.458282470703125
[step: 6] loss: 78.24053192138672
[step: 7] loss: 78.75645446777344
[step: 8] loss: 67.96855163574219
[step: 9] loss: 52.98954772949219
[step: 10] loss: 40.23509216308594
[step: 11] loss: 33.639503479003906
[step: 12] loss: 34.0942268371582
[step: 13] loss: 39.33544158935547
[step: 14] loss: 45.0288200378418
[step: 15] loss: 47.705806732177734
[step: 16] loss: 46.504112243652344
[step: 17] loss: 42.580596923828125
[step: 18] loss: 37.864356994628906
[step: 19] loss: 34.080078125
[step: 20] loss: 32.19560623168945
[step: 21] loss: 32.26582717895508
[step: 22] loss: 33.6153450012207
[step: 23] loss: 35.24607849121094
[step: 24] loss: 36.28779602050781
[step: 25] loss: 36.30300521850586
[step: 26] loss: 35.3519401550293
[step: 27] loss: 33.854400634765625
[step: 28] loss: 32.3569107055

[step: 229] loss: 18.346776962280273
[step: 230] loss: 18.709346771240234
[step: 231] loss: 18.21198272705078
[step: 232] loss: 18.537097930908203
[step: 233] loss: 18.141204833984375
[step: 234] loss: 18.347858428955078
[step: 235] loss: 18.068561553955078
[step: 236] loss: 18.182344436645508
[step: 237] loss: 17.97770881652832
[step: 238] loss: 18.046297073364258
[step: 239] loss: 17.86602020263672
[step: 240] loss: 17.937345504760742
[step: 241] loss: 17.748851776123047
[step: 242] loss: 17.842609405517578
[step: 243] loss: 17.63726806640625
[step: 244] loss: 17.74315643310547
[step: 245] loss: 17.554271697998047
[step: 246] loss: 17.61712646484375
[step: 247] loss: 17.514455795288086
[step: 248] loss: 17.464990615844727
[step: 249] loss: 17.486448287963867
[step: 250] loss: 17.35027503967285
[step: 251] loss: 17.3880558013916
[step: 252] loss: 17.32945442199707
[step: 253] loss: 17.24526596069336
[step: 254] loss: 17.278392791748047
[step: 255] loss: 17.215959548950195
[step: 256] 

[step: 458] loss: 15.579144477844238
[step: 459] loss: 15.571903228759766
[step: 460] loss: 15.570798873901367
[step: 461] loss: 15.569096565246582
[step: 462] loss: 15.577940940856934
[step: 463] loss: 15.589102745056152
[step: 464] loss: 15.626953125
[step: 465] loss: 15.674068450927734
[step: 466] loss: 15.81058406829834
[step: 467] loss: 15.939136505126953
[step: 468] loss: 16.330076217651367
[step: 469] loss: 16.35099220275879
[step: 470] loss: 16.69294548034668
[step: 471] loss: 16.070232391357422
[step: 472] loss: 15.707510948181152
[step: 473] loss: 15.497954368591309
[step: 474] loss: 15.606108665466309
[step: 475] loss: 15.890439987182617
[step: 476] loss: 15.89218521118164
[step: 477] loss: 15.811735153198242
[step: 478] loss: 15.546810150146484
[step: 479] loss: 15.480217933654785
[step: 480] loss: 15.601374626159668
[step: 481] loss: 15.717720031738281
[step: 482] loss: 15.798253059387207
[step: 483] loss: 15.632641792297363
[step: 484] loss: 15.49848461151123
[step: 485] 

[step: 686] loss: 14.959284782409668
[step: 687] loss: 14.957670211791992
[step: 688] loss: 14.957192420959473
[step: 689] loss: 14.960190773010254
[step: 690] loss: 14.975044250488281
[step: 691] loss: 15.022696495056152
[step: 692] loss: 15.205583572387695
[step: 693] loss: 15.629741668701172
[step: 694] loss: 17.38955307006836
[step: 695] loss: 16.772130966186523
[step: 696] loss: 17.559892654418945
[step: 697] loss: 15.113192558288574
[step: 698] loss: 16.702239990234375
[step: 699] loss: 20.116254806518555
[step: 700] loss: 16.291006088256836
[step: 701] loss: 23.04505157470703
[step: 702] loss: 21.90999984741211
[step: 703] loss: 24.107955932617188
[step: 704] loss: 20.1329402923584
[step: 705] loss: 19.7996768951416
[step: 706] loss: 20.002473831176758
[step: 707] loss: 21.34316635131836
[step: 708] loss: 19.286113739013672
[step: 709] loss: 18.196996688842773
[step: 710] loss: 20.3976993560791
[step: 711] loss: 16.97800064086914
[step: 712] loss: 17.21344566345215
[step: 713] l

In [5]:
# Plot predictions
plt.title(date + ' predict')
plt.plot(dataY2, label='raw data') # 데이터 있을때
plt.plot(test_predict, label='predict')
plt.xlabel("Time Period")
# plt.ylabel("photometric")
plt.ylabel("cct")
# plt.ylabel("CAS_SWR")
# plt.ylabel("446to477")
# plt.ylabel("UV-B")
plt.xticks(np.arange(0, 892, step = 56), ['5', '6', '7', '8', '9', '10', '11', '12', '1', '2', '3', '4', '5', '6', '7', '8']) # v2 모델
# plt.yticks(np.arange(0, 140000, step=10000)) # ptmt
plt.yticks(np.arange(0, 60000, step=10000)) # cct
# plt.yticks(np.arange(0, 2.0, step=0.2)) # uvb
# plt.ylim(0, 100) #swr, 446to477
plt.grid(True)
plt.legend(loc='upper right')
plt.show()